In [7]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Learning rate parameters
BASE_LR = 0.001
EPOCH_DECAY = 1 # number of epochs after which the Learning rate is decayed exponentially.
DECAY_WEIGHT = 0.1 # factor by which the learning rate is reduced.


# DATASET INFO
NUM_CLASSES = 4 # set the number of classes in your dataset

# DATALOADER PROPERTIES
BATCH_SIZE = 10 # Set as high as possible. If you keep it too high, you'll get an out of memory error.


### GPU SETTINGS
CUDA_DEVICE = 0 # Enter device ID of your gpu if you want to run on gpu. Otherwise neglect.
GPU_MODE = 0 # set to 1 if want to run on gpu.


# SETTINGS FOR DISPLAYING ON TENSORBOARD
USE_TENSORBOARD = 0 #if you want to use tensorboard set this to 1.
TENSORBOARD_SERVER = "YOUR TENSORBOARD SERVER ADDRESS HERE" # If you set.
EXP_NAME = "fine_tuning_experiment" # if using tensorboard, enter name of experiment you want it to be displayed as.


use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")
    
use_gpu = False


In [17]:
data_dir = 'IMAGES'

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
         for x in ['train', 'val']}
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=BATCH_SIZE,
                                               shuffle=True, num_workers=1)
                for x in ['train', 'val']}
dset_sizes = {x: len(dsets[x]) for x in ['train', 'val']}
dset_classes = dsets['train'].classes
dset_classes[2]

'Medium'

In [9]:
def train_model(model, criterion, optimizer, lr_scheduler, num_epochs=100):
    since = time.time()

    best_model = model
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                mode='train'
                optimizer = lr_scheduler(optimizer, epoch)
                model.train()  # Set model to training mode
            else:
                model.eval()
                mode='val'

            running_loss = 0.0
            running_corrects = 0

            counter=0
            # Iterate over data.
            for data in dset_loaders[phase]:
                inputs, labels = data
                print(inputs.size())
                # wrap them in Variable
                if use_gpu:
                    try:
                        inputs, labels = Variable(inputs.float().cuda()),                             
                        Variable(labels.long().cuda())
                    except:
                        print(inputs,labels)
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # Set gradient to zero to delete history of computations in previous epoch. Track operations so that differentiation can be done automatically.
                optimizer.zero_grad()
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                
                loss = criterion(outputs, labels)
                # print('loss done')                
                # Just so that you can keep track that something's happening and don't feel like the program isn't running.
                # if counter%10==0:
                #     print("Reached iteration ",counter)
                counter+=1

                # backward + optimize only if in training phase
                if phase == 'train':
                    # print('loss backward')
                    loss.backward()
                    # print('done loss backward')
                    optimizer.step()
                    # print('done optim')
                # print evaluation statistics
                try:
                    # running_loss += loss.data[0]
                    running_loss += loss.item()
                    # print(labels.data)
                    # print(preds)
                    running_corrects += torch.sum(preds == labels.data)
                    # print('running correct =',running_corrects)
                except:
                    print('unexpected error, could not calculate loss or do a sum.')
            print('trying epoch loss')
            epoch_loss = running_loss / dset_sizes[phase]
            epoch_acc = running_corrects.item() / float(dset_sizes[phase])
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))


            # deep copy the model
            if phase == 'val':
                if USE_TENSORBOARD:
                    foo.add_scalar_value('epoch_loss',epoch_loss,step=epoch)
                    foo.add_scalar_value('epoch_acc',epoch_acc,step=epoch)
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model = copy.deepcopy(model)
                    print('new best accuracy = ',best_acc)
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print('returning and looping back')
    return best_model

In [10]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = Variable(inputs.cuda(),volatile=True)
            labels = Variable(labels.cuda(),volatile=True)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [11]:
# This function changes the learning rate over the training model.
def exp_lr_scheduler(optimizer, epoch, init_lr=BASE_LR, lr_decay_epoch=EPOCH_DECAY):
    """Decay learning rate by a factor of DECAY_WEIGHT every lr_decay_epoch epochs."""
    lr = init_lr * (DECAY_WEIGHT**(epoch // lr_decay_epoch))

    if epoch % lr_decay_epoch == 0:
        print('LR is set to {}'.format(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer



In [13]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, NUM_CLASSES)


criterion = nn.CrossEntropyLoss()

if use_gpu:
    criterion.cuda()
    model_ft.cuda()

optimizer_ft = optim.RMSprop(model_ft.parameters(), lr=0.0001)



# Run the functions and save the best model in the function model_ft.
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=4)

# Save model


Epoch 0/3
----------
LR is set to 0.001
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])


torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size([10, 3, 224, 224])
torch.Size

In [14]:
torch.save(model_ft,"modelnew")